# DeLorean
## im quite tired atm tbh

### 1. Setup workspace

In [1]:
import pandas as pd
import numpy as np
import re

#Append folder for custom libraries
import sys
sys.path.append('libraries')

#Import custom library
import shinypanda

cars_price = pd.read_csv('data/cars_test.csv')

### 2. Explore the dataset

In [2]:
#Head of the dataframe
cars_price.head()

,Id,city,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,...,type,paint_color,lat,long,county_fips,county_name,state_fips,state_code,state_name,weather
0,974298,duluth,2006.0,ford,f-250 super duty,good,8 cylinders,gas,154400.0,clean,...,pickup,white,47.746524,-90.357742,27031.0,Cook,27.0,MN,Minnesota,43.0
1,1051884,kansascity,1987.0,chevrolet,NaN,NaN,NaN,gas,NaN,clean,...,NaN,NaN,38.373182,-93.776859,29083.0,Henry/Rives,29.0,MO,Missouri,52.0
2,684464,palmsprings,2010.0,jeep,liberty sport,NaN,6 cylinders,gas,127722.0,clean,...,SUV,NaN,33.741059,-116.356434,6065.0,Riverside,6.0,CA,California,59.0
3,1255387,sanmarcos,2003.0,chevrolet,tahoe,fair,8 cylinders,gas,NaN,clean,...,SUV,white,30.026266,-98.133363,48209.0,Hays,48.0,TX,Texas,67.0
4,1195520,tampa,2006.0,lexus,gs 300,NaN,NaN,gas,NaN,clean,...,NaN,NaN,27.868800,-82.734400,12103.0,Pinellas,12.0,FL,Florida,65.0


In [3]:
cars_price.count()

Id              253073
city            253073
year            252162
manufacturer    233025
make            242961
condition       150378
cylinders       151927
fuel            251490
odometer        169963
title_status    252700
transmission    251723
drive           156207
size             88234
type            150050
paint_color     151286
lat             253073
long            253073
county_fips     244441
county_name     244441
state_fips      244441
state_code      244441
state_name      253073
weather         244357
dtype: int64

In [4]:
#Type of data
#cars_price.dtypes

In [5]:
#Descriptive stats
#cars_price.describe()

### 3. Data cleaning
#### In order to use sklearn algorithms, the entire dataset must contain operable numerical data

#### 3.1 Setup dataframe

In [6]:
#Remove all location info but State Code (could affect to taxes)
location_columns = ['city', 'lat', 'long', 'county_fips', 'county_name', 'state_fips', 'state_name']
cars_price = cars_price.drop(columns=location_columns, axis=1)

#Remove 'Id' column (does not provide info)
cars_price = cars_price.drop(columns=['Id'])

#Remove unpopulated columns
cars_price = cars_price.drop(columns=['drive', 'size', 'type', 'paint_color', 'condition', 'cylinders', 'odometer'], axis=1)

#Rename columns
cars_price.rename(columns={'manufacturer':'brand',
                          'make':'model',
                          'state_code':'state'}, 
                          inplace=True)

cars_price.head()

,year,brand,model,fuel,title_status,transmission,state,weather
0,2006.0,ford,f-250 super duty,gas,clean,automatic,MN,43.0
1,1987.0,chevrolet,NaN,gas,clean,automatic,MO,52.0
2,2010.0,jeep,liberty sport,gas,clean,automatic,CA,59.0
3,2003.0,chevrolet,tahoe,gas,clean,automatic,TX,67.0
4,2006.0,lexus,gs 300,gas,clean,automatic,FL,65.0


#### 3.2 Clean 'brand' column

In [7]:
#Transform column type from 'object' to string
cars_price['brand'] = cars_price['brand'].astype('str') 

brands = sorted(cars_price.brand.unique())
print('Original brands list:', brands)

Original brands list: ['acura', 'alfa', 'alfa-romeo', 'aston', 'aston-martin', 'audi', 'bmw', 'buick', 'cadillac', 'chev', 'chevrolet', 'chevy', 'chrysler', 'datsun', 'dodge', 'ferrari', 'fiat', 'ford', 'gmc', 'harley', 'harley-davidson', 'honda', 'hyundai', 'infiniti', 'infinity', 'jaguar', 'jeep', 'kia', 'land rover', 'landrover', 'lexus', 'lincoln', 'mazda', 'mercedes', 'mercedes-benz', 'mercedesbenz', 'mercury', 'mini', 'mitsubishi', 'morgan', 'nan', 'nissan', 'pontiac', 'porche', 'ram', 'rover', 'saturn', 'subaru', 'toyota', 'volkswagen', 'volvo', 'vw']


In [8]:
#Remove NaN
cars_price['brand'] = cars_price.brand.replace({r'nan': ''}, regex=True)

#Fix duplicity: 'alfa' and 'alfa-romeo'
cars_price['brand'] = cars_price.brand.replace({r'alfa-romeo': 'alfa'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'alfa': 'alfa-romeo'}, regex=True)

#Fix duplicity: 'aston' and 'aston-martin'
cars_price['brand'] = cars_price.brand.replace({r'aston-martin': 'aston'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'aston': 'aston-martin'}, regex=True)

#Fix duplicity: 'chev', 'chevy' and 'aston-martin'
cars_price['brand'] = cars_price.brand.replace({r'chevrolet': 'chev'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'chevy': 'chev'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'chev': 'chevrolet'}, regex=True)

#Fix duplicity: 'harley' and 'harley-davidson'
cars_price['brand'] = cars_price.brand.replace({r'harley-davidson': 'harley'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'harley': 'harley-davidson'}, regex=True)

#Fix misspelling: 'infinity' instead of 'infinity'
cars_price['brand'] = cars_price.brand.replace({r'infinity': 'infiniti'}, regex=True)

#Fix duplicity: 'rover', 'landrover' and 'land rover'
cars_price['brand'] = cars_price.brand.replace({r'landrover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'land rover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'rover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'land-rovex': 'land-rover'}, regex=True)

#Fix duplicity: 'landrover' and 'land rover'
cars_price['brand'] = cars_price.brand.replace({r'mercedes-benz': 'mercedes'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'mercedes': 'mercedes-benz'}, regex=True)

#Fix misspelling: 'porsche' instead of 'porche'
cars_price['brand'] = cars_price.brand.replace({r'porche': 'porsche'}, regex=True)

#Fix duplicity: 'vw' and 'volkswagen'
cars_price['brand'] = cars_price.brand.replace({r'vw': 'volkswagen'}, regex=True)

brands = sorted(cars_price.brand.unique())
print('Clean brands list:', brands)

Clean brands list: ['', 'acura', 'alfa-romeo', 'aston-martin', 'audi', 'bmw', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'datsun', 'dodge', 'ferrari', 'fiat', 'ford', 'gmc', 'harley-davidson', 'honda', 'hyundai', 'infiniti', 'jaguar', 'jeep', 'kia', 'land-rover', 'lexus', 'lincoln', 'mazda', 'mercedes-benz', 'mercedes-benzbenz', 'mercury', 'mini', 'mitsubishi', 'morgan', 'nissan', 'pontiac', 'porsche', 'ram', 'saturn', 'subaru', 'toyota', 'volkswagen', 'volvo']


In [9]:
#Import new brands (scraped from Wikipedia using brands-importer.ipynb)
new_brands = pd.read_csv('data/brands.csv')
new_brands.head()

,Unnamed: 0,name
0,0,adelmo
1,1,asa
2,2,de-carlo
3,3,feresa
4,4,koller


In [10]:
#Convert it into a list and merge both lists
new_brands = list(new_brands['name'])
brands = list(set(brands + new_brands))

#### 3.3 Clean 'model' column

In [11]:
print('There are', len(cars_price['model'].unique()), 'different models in the first dataset')

There are 31070 different models in the first dataset


In [12]:
#Transform column type from 'object' to string
cars_price['model'] = cars_price['model'].astype('str')

#Make it lowercase
cars_price['model'] = cars_price['model'].str.lower()

cars_price.head()

,year,brand,model,fuel,title_status,transmission,state,weather
0,2006.0,ford,f-250 super duty,gas,clean,automatic,MN,43.0
1,1987.0,chevrolet,nan,gas,clean,automatic,MO,52.0
2,2010.0,jeep,liberty sport,gas,clean,automatic,CA,59.0
3,2003.0,chevrolet,tahoe,gas,clean,automatic,TX,67.0
4,2006.0,lexus,gs 300,gas,clean,automatic,FL,65.0


In [13]:
#Fix misspelling
cars_price['model'] = cars_price.model.replace({r'chervolet': 'chevrolet'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'volksvagen': 'volkswagen'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'mercedes-benzbenz': 'mercedes-benz'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'nissaan': 'nissan'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'totota': 'toyota'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'suburu': 'subaru'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'crystler': 'chrysler'}, regex=True)

In [14]:
#Split 'model' data
cars_price['extracted_brand'] = cars_price["model"].str.split(" ", n=-1, expand=False) 

#When 'model' contains a string that already exists in 'brands', export it in a new column
def brand_in_model(row):
    
    brand = list(set(row['extracted_brand']).intersection(brands))
    return brand

#Apply the function and convert column to string again
cars_price['extracted_brand'] = cars_price.apply(lambda row: brand_in_model(row), axis=1)
cars_price['extracted_brand'] = cars_price['extracted_brand'].apply(lambda brand: ','.join(map(str, brand)))

cars_price.head()

,year,brand,model,fuel,title_status,transmission,state,weather,extracted_brand
0,2006.0,ford,f-250 super duty,gas,clean,automatic,MN,43.0,
1,1987.0,chevrolet,nan,gas,clean,automatic,MO,52.0,
2,2010.0,jeep,liberty sport,gas,clean,automatic,CA,59.0,
3,2003.0,chevrolet,tahoe,gas,clean,automatic,TX,67.0,
4,2006.0,lexus,gs 300,gas,clean,automatic,FL,65.0,


In [15]:
#Split 'extracted brand' to preserve only the first match with brands list
split_brand = cars_price['extracted_brand'].str.split(',', n=1, expand=True) 
cars_price['extracted_brand'] = split_brand[0]

print('New brands found in \'model\' column:', sorted(cars_price['extracted_brand'].unique()))

New brands found in 'model' column: ['', 'abarth', 'ac', 'aero', 'amc', 'astra', 'ata', 'austin', 'austin-healey', 'autozam', 'bentley', 'bond', 'bricklin', 'bugatti', 'bullet', 'canadian', 'cap', 'checker', 'chevrolet', 'chevron', 'chrysler', 'cla', 'cobra', 'colt', 'crosley', 'ct&t', 'daewoo', 'daihatsu', 'daytona', 'desoto', 'dr', 'ds', 'e-z-go', 'eagle', 'edsel', 'envoy', 'falcon', 'genesis', 'geo', 'healey', 'hino', 'hudson', 'humber', 'hummer', 'imperial', 'innocenti', 'isuzu', 'jensen', 'kaiser', 'karma', 'kawasaki', 'lada', 'lamborghini', 'lasalle', 'local', 'lotus', 'marauder', 'maserati', 'master', 'maybach', 'mclaren', 'mclaughlin', 'merkur', 'meteor', 'metropolitan', 'mg', 'miura', 'monarch', 'moon', 'morris', 'mustang', 'nash', 'oldsmobile', 'opel', 'packard', 'panther', 'peugeot', 'pioneer', 'plymouth', 'polaris', 'porsche', 'premier', 'princess', 'puma', 'rambler', 'ranger', 'redline', 'renault', 'rolls-royce', 'rw', 'saab', 'saleen', 'scion', 'seat', 'smart', 'standard'

In [16]:
cars_price.head()

,year,brand,model,fuel,title_status,transmission,state,weather,extracted_brand
0,2006.0,ford,f-250 super duty,gas,clean,automatic,MN,43.0,
1,1987.0,chevrolet,nan,gas,clean,automatic,MO,52.0,
2,2010.0,jeep,liberty sport,gas,clean,automatic,CA,59.0,
3,2003.0,chevrolet,tahoe,gas,clean,automatic,TX,67.0,
4,2006.0,lexus,gs 300,gas,clean,automatic,FL,65.0,


In [17]:
#Where 'brand' is not available, use 'extracted_brand'
def complete_brand(row):
    
    brand_doesnt_exist = row['brand']==''
    extracted_brand_exists = row['extracted_brand']!=''
    
    if brand_doesnt_exist and extracted_brand_exists:
        return row['extracted_brand']
    else:
        return row['brand']

cars_price['brand'] = cars_price.apply(lambda row: complete_brand(row), axis=1)

#Remove 'extracted_brand' and 'model' columns
cars_price = cars_price.drop(['model', 'extracted_brand'], axis=1)

In [18]:
cars_price.head()

,year,brand,fuel,title_status,transmission,state,weather
0,2006.0,ford,gas,clean,automatic,MN,43.0
1,1987.0,chevrolet,gas,clean,automatic,MO,52.0
2,2010.0,jeep,gas,clean,automatic,CA,59.0
3,2003.0,chevrolet,gas,clean,automatic,TX,67.0
4,2006.0,lexus,gas,clean,automatic,FL,65.0


### 3. Make model

In [19]:
cars_price.dtypes

year            float64
brand            object
fuel             object
title_status     object
transmission     object
state            object
weather         float64
dtype: object

In [20]:
cars_price['brand'] = cars_price['brand'].astype('str')
cars_price['fuel'] = cars_price['fuel'].astype('str') 
cars_price['title_status'] = cars_price['title_status'].astype('str') 
cars_price['transmission'] = cars_price['transmission'].astype('str') 
cars_price['state'] = cars_price['state'].astype('str') 

In [21]:
cars_price.dtypes

year            float64
brand            object
fuel             object
title_status     object
transmission     object
state            object
weather         float64
dtype: object

In [22]:
year_values = sorted(cars_price['year'].unique())
brand_values = sorted(cars_price['brand'].unique())
fuel_values = cars_price['fuel'].unique()
title_status = sorted(cars_price['title_status'].unique())
transmission_values = sorted(cars_price['transmission'].unique())
state_values = sorted(cars_price['state'].unique())
weather_values = sorted(cars_price['weather'].unique())

In [23]:
cars_price = cars_price.replace(r'^\s*$', np.nan, regex=True)

In [24]:
#cars_price = cars_price.dropna()

In [25]:
cars_price.count()

year            252162
brand           238913
fuel            253073
title_status    253073
transmission    253073
state           253073
weather         244357
dtype: int64

In [26]:
cars_price = cars_price.fillna(method='ffill')

In [27]:
final_dataset = pd.get_dummies(cars_price)

In [28]:
final_dataset.head()

,year,weather,brand_ac,brand_acura,brand_aero,brand_alfa-romeo,brand_amc,brand_aston-martin,brand_ata,brand_audi,...,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,state_nan
0,2006.0,43.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1987.0,52.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2010.0,59.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2003.0,67.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2006.0,65.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
only_brands = cars_price.drop('state', axis=1)
only_brands = pd.get_dummies(only_brands)

In [30]:
only_brands.head()

,year,weather,brand_ac,brand_acura,brand_aero,brand_alfa-romeo,brand_amc,brand_aston-martin,brand_ata,brand_audi,...,title_status_lien,title_status_missing,title_status_nan,title_status_parts only,title_status_rebuilt,title_status_salvage,transmission_automatic,transmission_manual,transmission_nan,transmission_other
0,2006.0,43.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1987.0,52.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2010.0,59.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,2003.0,67.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,2006.0,65.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [34]:
test_dataset = only_brands.copy()
training_dataset = pd.read_csv('data/trained_dataframe.csv')

In [35]:
#In training_dataset, but not in test
training_diff = training_dataset.columns.difference(test_dataset.columns)
training_diff

Index(['brand_alpine', 'brand_astra', 'brand_auburn', 'brand_blade',
       'brand_borgward', 'brand_bullet', 'brand_cheetah', 'brand_cla',
       'brand_cmc', 'brand_colt', 'brand_cord', 'brand_daf', 'brand_daimler',
       'brand_direct', 'brand_enfield', 'brand_fab', 'brand_frazer',
       'brand_gaz', 'brand_glas', 'brand_goggomobil', 'brand_hillman',
       'brand_invicta', 'brand_jowett', 'brand_karma', 'brand_lafayette',
       'brand_lancia', 'brand_lloyd', 'brand_maxwell', 'brand_mclaughlin',
       'brand_mercedes-amg', 'brand_meteor', 'brand_mine', 'brand_minerva',
       'brand_nsu', 'brand_panoz', 'brand_pininfarina', 'brand_reliant',
       'brand_siata', 'brand_singer', 'brand_tac', 'brand_tornado',
       'brand_tvr', 'brand_walter', 'price'],
      dtype='object')

In [36]:
for column in training_diff[:-1]:
    test_dataset[column] = 0

In [37]:
#In test_dataset, but not in training
test_diff = test_dataset.columns.difference(training_dataset.columns)
test_diff

Index(['brand_ata', 'brand_bond', 'brand_canadian', 'brand_ct&t',
       'brand_humber', 'brand_mercedes-benzbenz', 'brand_miura',
       'brand_monarch', 'brand_panther', 'brand_talbot'],
      dtype='object')

In [38]:
for column in test_diff:
    training_dataset[column] = 0

In [39]:
training_dataset.shape

(426861, 194)

In [40]:
test_dataset.shape

(253073, 193)

In [45]:
from sklearn import linear_model

x_columns = [col for col in training_dataset.columns.values if col != 'price'] 

cars_x = training_dataset[x_columns] 
cars_y = training_dataset['price']

cars_model = linear_model.LinearRegression() 
cars_model.fit(cars_x, cars_y)
cars_model.score(cars_x, cars_y)

0.00022158460356391796

In [42]:
final_result = cars_model.predict(test_dataset)

In [43]:
dataframe_to_send = pd.read_csv('data/cars_test.csv')

dataframe_to_send['price'] = final_result
dataframe_to_send = dataframe_to_send[['Id', 'price']]